## OBJECTIVE'S 
- **understand the structure of the full G1 MuJoCo model by loading it, listing joints and actuators** 
- **noting what needs to be mapped before applying control later**

### Sub Tasks
- load G1 XML in notebook
- print joint names and actuator names
- write brief notes on joint count and naming style
- identify 1 joint to target and train that in gymasium

#### Documentaiton Refered
- https://github.com/Sentdex/unitree_g1_vibes/blob/main/RL-shenanigans/train_g1_arm_policy.py
- https://github.com/google-deepmind/dm_control/blob/3a1fec1090989abe5a766238a38b64802708abd8/README.md
- https://deepmind.google/blog/dm-control-software-and-tasks-for-continuous-control/

#### Youtube video referd
- https://www.youtube.com/watch?v=wiIUF9pIDYw


#### Check if dm_contorl and Mujoco are correctly installed

In [29]:
print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError('dn_contorl is not installed on the system!')
else:
  del suite, pixels
print('dm_control installation succeeded.')
     

Checking that the dm_control installation succeeded...
dm_control installation succeeded.


#### All `dm_control` imports required during trails/experiments

In [30]:
# The basic mujoco wrapper.
from dm_control import mujoco

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib


#### Other necassory import

In [44]:
import numpy as np
from dm_control import mujoco
from dm_control import mujoco as dm_mujoco
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import PIL.Image
import itertools
from IPython.display import clear_output

In [45]:
import os
from pathlib import Path

# project_root = Path(".").resolve().parent     #if notebook location at root
project_root =Path("..").resolve()              #if notebook location inside subfolder
xml_path = str(project_root / "models/G1/g1_29dof_rev_1_0.xml")

### Example : G1 

In [46]:
# Loading Model
print(xml_path)
physics = mujoco.Physics.from_xml_path(xml_path)

/home/blank/Projects/Internship/sentienc/Robotic-Simulation/robot_ws/mujoco-experiments/models/G1/g1_29dof_rev_1_0.xml


#### List for Joint's and Actuator's 

In [47]:
# Print all Joints names and their IDs
joint_name_list = []
for i in range(physics.model.njnt):
    # Print the joint's name
    joint_name = physics.model.id2name(i, 'joint')
    joint_name_list.append(joint_name)
    print(f"\nJoint ID {i}: {joint_name}")
clear_output()

In [48]:
# Print all Actuator names and their IDs
actuator_name_list = []
for i in range(physics.model.nu):
    actuator_name = physics.model.id2name(i, 'actuator')
    actuator_name_list.append(actuator_name)
    print(f"Actuator ID: {i} | Name: {actuator_name}")
clear_output()

In [ ]:
# We start from 1 as 0th index contain a base joint which dosent have a pairing actuator 
actuator_name_array = np.array(actuator_name_list)
joint_name_array = np.array(joint_name_list[1:])
combined_pair = list(zip(joint_name_list[1:], actuator_name_list))

In [57]:
print(combined_pair)

[('left_hip_pitch_joint', 'left_hip_pitch_joint'), ('left_hip_roll_joint', 'left_hip_roll_joint'), ('left_hip_yaw_joint', 'left_hip_yaw_joint'), ('left_knee_joint', 'left_knee_joint'), ('left_ankle_pitch_joint', 'left_ankle_pitch_joint'), ('left_ankle_roll_joint', 'left_ankle_roll_joint'), ('right_hip_pitch_joint', 'right_hip_pitch_joint'), ('right_hip_roll_joint', 'right_hip_roll_joint'), ('right_hip_yaw_joint', 'right_hip_yaw_joint'), ('right_knee_joint', 'right_knee_joint'), ('right_ankle_pitch_joint', 'right_ankle_pitch_joint'), ('right_ankle_roll_joint', 'right_ankle_roll_joint'), ('waist_yaw_joint', 'waist_yaw_joint'), ('waist_roll_joint', 'waist_roll_joint'), ('waist_pitch_joint', 'waist_pitch_joint'), ('left_shoulder_pitch_joint', 'left_shoulder_pitch_joint'), ('left_shoulder_roll_joint', 'left_shoulder_roll_joint'), ('left_shoulder_yaw_joint', 'left_shoulder_yaw_joint'), ('left_elbow_joint', 'left_elbow_joint'), ('left_wrist_roll_joint', 'left_wrist_roll_joint'), ('left_wrist_

### Final Conclusion and Learning form this 

In this notebook, we transitioned from basic physics simulation to the foundations of Robot Learning. Here is the summary of our key takeaways:
1. The "Body" (MuJoCo XML)

    Hierarchy is everything: We learned that joints and geoms must be nested within bodies to move correctly.

    The Actuator Gap: We discovered that a joint is just a "pivot," but a <motor> is the "muscle." Without naming actuators, we cannot easily target them via code.

2. The "Nervous System" (dm_control)

    Physics vs. Data: We separated physics.model (the static rules) from physics.data (the dynamic state like time, velocity, and control signals).

    Named Accessors: We moved from using mysterious indices like ctrl[0] to descriptive names like ctrl['swing_motor'], making the code modular and readable.

3. The "Brain" or the Control Logic

    Open-Loop Control: we implemented a Sine Wave controller to create rhythmic "to-and-fro" motion in previous notebook.

    `The Control Loop: We mastered the flow: Calculate Command → Apply to Actuator → Step Physics → Render Frame.`

4. Scaling to RL (Gymnasium)

    The Wrapper Concept: We realized that dm_control provides the "truth" of physics, while Gymnasium provides the "rules of the game" (Rewards and Observations) required for AI training.
    
    
###  So we will shifting our focus to traning in Gymnasium as we have learned the dm_Control will be mainly used for debugging and fast paced testing stuff